In [1]:
import matplotlib.pyplot as plt
import phoebe as pb
from phoebe import u
import pandas as pd 

PHOEBE: passband "Bolometric:900-40000" has a newer version available.  Run phoebe.list_passband_online_history("Bolometric:900-40000") to get a list of available changes and phoebe.update_passband("Bolometric:900-40000") or phoebe.update_all_passbands() to update.
PHOEBE: passband "Johnson:V" has a newer version available.  Run phoebe.list_passband_online_history("Johnson:V") to get a list of available changes and phoebe.update_passband("Johnson:V") or phoebe.update_all_passbands() to update.


In [2]:
logger = pb.logger()
b = pb.default_binary()
b.add_dataset('lc', times=pb.linspace(0,1,101), dataset='lc01', overwrite=True)

<ParameterSet: 80 parameters | contexts: compute, figure, constraint, dataset>

In [3]:
b["ecc@binary"] = 0.26
# b["q@binary"] = 0.17
b["q@binary"] = 0.18
b["sma@binary"] = 40 * u.solRad
b["incl@binary"] = 79.5
b["per0"] = 275 *u.deg
b["period@binary"] = 5.9525 *u.day
b['t0_supconj@component'] = 1629


#primary
# b["mass@primary@component"] = 13.5
# b.flip_constraint("sma@binary", solve_for="mass")
b["requiv@primary"] = 8.8 #2013: 8.8; 
b["teff@primary"] = 25000
b["gravb_bol@primary"] = 1.0
b["irrad_frac_refl_bol@primary"] = 1.0
# b.flip_constraint("logg@primary", solve_for="requiv@primary")
# b["logg@primary@star@component"] = 3.8
# b.flip_constraint("requiv@primary",solve_for="logg@primary")

#secondary
# b.flip_constraint("mass@secondary", solve_for="q")
# b["mass@secondary@component"] = 1.8

# b.flip_constraint("q@binary", solve_for="mass")
b["requiv@secondary@component"] = 1.3 #1.5+/-0.2
# b['teffratio@binary'] = 0.21
b["teff@secondary@component"] = 6000

b["gravb_bol@secondary"] = 0.7
b["irrad_frac_refl_bol@secondary"] = 0.7

In [4]:
b.add_dataset('lc', times=pb.linspace(0,1,101), dataset='lc01', overwrite=True)
# b.add_dataset('rv', times=pb.linspace(0,1,101), dataset='rv01', overwrite=True)

b.run_compute()

Sun, 09 Feb 2025 19:13 BUNDLE       WARNING ld_mode_bol of 'lookup' uses a bolometric passband which is not reliable for hot stars.  Consider using ld_mode_bol of manual and providing ld_coeffs instead.
100%|██████████| 101/101 [00:01<00:00, 61.35it/s]


<ParameterSet: 3 parameters | qualifiers: times, fluxes, comments>

In [5]:
plotter = b.plot(x='phase',  ylabel = 'Normalized Flux', xlabel = 'Phase', show=True, save = 'pbtest.png')
# plt.ion()
plt.show(block=False)

/opt/anaconda3/envs/phoebe_environment/lib/python3.13/site-packages/phoebe/dependencies/autofig/figure.py:627: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()  # <-- blocking
/var/folders/34/3847lqd14j78mfm70_z1c_r00000gn/T/ipykernel_13971/3563954529.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show(block=False)
